In [52]:
import os 
import pandas as pd
import numpy as np
import h5py
from numpy import nan
from datetime import datetime, timedelta
from time import mktime
import os.path


checkpoint_name = './checkpoints/data_2.h5'
df = pd.read_hdf(checkpoint_name,'table')

df.head()

,entryType,entrySubType,eventType,rampTime,rampBaseValue,rampFinalValue,signal event ID,signal Value,Signal Name,Signal Value string,Signal direction,unix_time,room_name_merged,message_merged,device_id_name,temperature,heat_setpoint,cool_setpoint,auto_setpoint
1,Lights,Scene,SceneChangedEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,SceneRecalled,74Night,NaN,NaN,NaN,NaN
3,DoorLock,Device,SignalChangedEventWithStrings,NaN,NaN,NaN,NaN,NaN,Basement Mudroom,Lock,ToDevice,1.502438e+09,NaN,NaN,208Basement Mudroom Door,NaN,NaN,NaN,NaN
4,DoorLock,Scene,SceneChangedEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Basement Mudroom,SceneRecalled,77Lock Basement Mudroom Door,NaN,NaN,NaN,NaN
13,Lights,Load,LevelChangedEvent,0,0,19275,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,NaN,118Sconces,NaN,NaN,NaN,NaN
14,Lights,Load,LevelChangedEvent,-1,0,0,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,NaN,122Chandelier,NaN,NaN,NaN,NaN


In [69]:
def check_signal_direction(df, string_val):
    return df[df["Signal Value string"] == string_val]['Signal direction'].unique()

In [6]:
d = {}
for i in sorted(df["Signal Value string"].astype(str).unique()):
    d[i] = check_signal_direction(df, i)

In [14]:
for k,v in d.items():
    print(k, v)

"Leave" Event ['Heat Setpoint 68° Fahrenheit, Cool Setpoint 74° Fahrenheit, and Auto Setpoint 72° Fahrenheit']
"Return" Event ['Heat Setpoint 71° Fahrenheit, Cool Setpoint 74° Fahrenheit, and Auto Setpoint 72° Fahrenheit']
"Sleep" Event ['Heat Setpoint 67° Fahrenheit, Cool Setpoint 77° Fahrenheit, and Auto Setpoint 72° Fahrenheit'
 'Heat Setpoint 67° Fahrenheit, Cool Setpoint 72° Fahrenheit, and Auto Setpoint 72° Fahrenheit'
 'Heat Setpoint 67° Fahrenheit, Cool Setpoint 71° Fahrenheit, and Auto Setpoint 73° Fahrenheit'
 'Heat Setpoint 67° Fahrenheit, Cool Setpoint 73° Fahrenheit, and Auto Setpoint 73° Fahrenheit']
"Wake Weekend" Event ['Heat Setpoint 71° Fahrenheit, Cool Setpoint 74° Fahrenheit, and Auto Setpoint 70° Fahrenheit']
"Wake" Event ['Heat Setpoint 71° Fahrenheit, Cool Setpoint 75° Fahrenheit, and Auto Setpoint 72° Fahrenheit'
 'Heat Setpoint 71° Fahrenheit, Cool Setpoint 74° Fahrenheit, and Auto Setpoint 72° Fahrenheit']
"Weekend Wake" Event ['Heat Setpoint 71° Fahrenheit, C

In [53]:
## d[x] = [val_strings*]
d = {'Active Fan Level' : ['High','Off'],
'Auto Mode': ['Enabled', 'Disabled'],
'Cool Stage 1' : ['Active', 'Inactive'],
'Fan' : ['On', 'Auto'],
'Floor Warming' : ['Heat:False','Off:False', 'Off:True'],
'Heat Stage 1' : ['Active', 'Inactive'],
'Hold' : ['On', 'Off'],
'Humidifier Off' : ['Inactive', 'Active'],
'Mode' : ['Heat' 'Cool' 'Off']}

In [56]:
def is_binary(row, **kwargs):
    """kwargs = [Signal Value, values_dict]"""
    if str(row['eventType']) == 'SignalChangedEventWithStrings':
#         import pdb; pdb.set_trace()
        for key, values in kwargs.items():
            if str(row['Signal Value string']) == key:
                for val in values:
                    if str(row['Signal direction']) == val:
                        return key + val
    return np.nan

In [66]:
df['event'] = df.apply(is_binary, **d, axis=1)

In [70]:
df['message_merged'].unique()

array(['SceneRecalled', nan, 'FromDevice', 'OccupancyDetected',
       'VacancyDetected', 'ToDevice', 'GraceOccupancyDetected',
       'Cause: Key or Thumb-turn.', 'Cause: Keypad PIN Entry.',
       'SceneFastRecalled', 'Cause: LockUnlockMalfunction',
       'Cause: One-touch.'], dtype=object)

In [68]:
df.head()

,entryType,entrySubType,eventType,rampTime,rampBaseValue,rampFinalValue,signal event ID,signal Value,Signal Name,Signal Value string,Signal direction,unix_time,room_name_merged,message_merged,device_id_name,temperature,heat_setpoint,cool_setpoint,auto_setpoint,event
1,Lights,Scene,SceneChangedEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,SceneRecalled,74Night,NaN,NaN,NaN,NaN,NaN
3,DoorLock,Device,SignalChangedEventWithStrings,NaN,NaN,NaN,NaN,NaN,Basement Mudroom,Lock,ToDevice,1.502438e+09,NaN,NaN,208Basement Mudroom Door,NaN,NaN,NaN,NaN,NaN
4,DoorLock,Scene,SceneChangedEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Basement Mudroom,SceneRecalled,77Lock Basement Mudroom Door,NaN,NaN,NaN,NaN,NaN
13,Lights,Load,LevelChangedEvent,0,0,19275,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,NaN,118Sconces,NaN,NaN,NaN,NaN,NaN
14,Lights,Load,LevelChangedEvent,-1,0,0,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,NaN,122Chandelier,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Keeping: device_id_merged, room_name_merged, regression_value_type, value, event, time